**Import required Python modules**

In [62]:
from whisper_jax import FlaxWhisperPipline
import jax.numpy as jnp
import time
import os

**CAN TAKE A WHILE: Instantiate pipeline with settings fitting to A100 grpahics card: bfloat16 and batching enabled**

In [3]:
pipeline = FlaxWhisperPipline("openai/whisper-large-v2", dtype=jnp.bfloat16, batch_size=16)

**Search for files in "input" -> check the response to see if files have been found!**

In [110]:

# search for the files in folder "input" that are not hidden
path = "input"
all_files = [f for f in os.listdir(path) if not f.startswith('.')]
amount = len(all_files)

# print some stuff for debugging
print("found " + str(amount) + " files: \n\n----------\n" + str(all_files))
print("----------")
if amount > 1:
    print("\nFound multiple files, only use first file: " + all_files[0])
    input_file = all_files[0]
if amount == 1:
    print("\nFound one file: " + all_files[0])
    input_file = all_files[0]
if amount == 0:
    print("ERROR: NO FILE FOUND")
    input_file = NULL


found 1 files: 

----------
['audio.wav']
----------

Found one file: audio.wav


In [134]:
# save current time
t = time.time()
# do the actual transcription of the FRIST file in folder "input". No language is defined, so Whisper will auto-detect the language.
outputs = pipeline(path + "/" + input_file,  task="transcribe", return_timestamps=True)
# print out the time needed for the transcription
print("Transcription needed " +  str(int(time.time() - t)) + " seconds:")
print("--------------------------------------\n\n" + outputs['text'].lstrip())

Transcription needed 14 seconds:
--------------------------------------

Ich nehme jetzt noch mal den Text auf, der erst auf Deutsch ist. And I will switch to English while talking, testing the pre-settings, if it's better to just keep it open or to define the language or even what happens, wenn man erst Deutsch auswählt und dann aber trotzdem Englisch spricht. you


**save transcription as coninuous text and with timestamps (mm:ss format) in the "output" folder**

In [59]:
with open('output/timestamped.txt', 'w') as file:
    for chunk in outputs['chunks']:
        minutes, seconds = divmod(int(chunk['timestamp'][0]), 60)
        file.write(f'{minutes:02d}:{seconds:02d}\n')
        file.write(chunk['text'].lstrip() + "\n\n")  # .lstrip() entfernt führende Leerzeichen

with open('output/continuous.txt', 'w') as file:
    file.write(outputs['text'].lstrip())

**the end**